---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

import pandas as pd
import numpy as np

df_test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
df_train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
df_address =  pd.read_csv('addresses.csv')
df_latlons = pd.read_csv('latlons.csv')

df_address = df_address.set_index('address').join(df_latlons.set_index('address'), how='left').reset_index()
df_train = df_train.set_index('ticket_id').join(df_address.set_index('ticket_id')).reset_index()
df_test = df_test.set_index('ticket_id').join(df_address.set_index('ticket_id')).reset_index()


df_train = df_train[~df_train.compliance.isnull()]
df_train = df_train[~df_train.hearing_date.isnull()]
df_train['date_diff'] = abs((pd.to_datetime(pd.Series(df_train.ticket_issued_date)).dt.date - \
    pd.to_datetime(pd.Series(df_train.hearing_date)).dt.date )).fillna(pd.Timedelta('-1 days')).dt.days

df_test['date_diff'] = abs((pd.to_datetime(pd.Series(df_test.ticket_issued_date)).dt.date - \
    pd.to_datetime(pd.Series(df_test.hearing_date)).dt.date )).fillna(pd.Timedelta('-1 days')).dt.days

df_train.lat.fillna(method='pad', inplace=True)
df_train.lon.fillna(method='pad', inplace=True)
df_train.state.fillna(method='pad', inplace=True)

df_test.lat.fillna(method='pad', inplace=True)
df_test.lon.fillna(method='pad', inplace=True)
df_test.state.fillna(method='pad', inplace=True)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df = pd.concat([df_train['agency_name'], df_test['agency_name']], ignore_index=True)
le.fit(df)
df_train['a_l'] = le.transform(df_train['agency_name'])
df_test['a_l'] = le.transform(df_test['agency_name'])
le2 = preprocessing.LabelEncoder()
df = pd.concat([df_train['city'], df_test['city']]).fillna(method='backfill')
le2.fit(df)
df_train['c_l'] = le2.transform(df_train['city'])
df_test['c_l'] = le2.transform(df_test['city'].fillna(method='backfill'))
df_train['violator_name'].fillna(value='ML',inplace=True)
df_test['violator_name'].fillna(value='ML',inplace=True)
df_train['zip_code_c'] = pd.to_numeric(df_train.zip_code, errors='coerce').fillna(method='backfill')
df_test['zip_code_c'] = pd.to_numeric(df_test.zip_code, errors='coerce').fillna(method='backfill')   
target_column = ['compliance']

selected_features = ['fine_amount', 'admin_fee', 'state_fee',
               'late_fee','clean_up_cost','discount_amount','judgment_amount', 'date_diff',
                'violation_street_number','a_l','c_l','zip_code_c'
                     ,'agency_name', 'state', 'disposition'
               ]

X = df_train[selected_features]
X = pd.get_dummies(X, columns=['agency_name', 'state', 'disposition'])
y = df_train[target_column]


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                   random_state = 0)

clf = RandomForestClassifier(n_estimators=200).fit(X, y)

test = df_test[selected_features]
#test = pd.get_dummies(test, columns=['agency_name', 'state', 'disposition'])

test = scaler.transform(test)

result = clf.predict_proba(test)

selected_features = ['fine_amount', 'admin_fee', 'state_fee',
               'late_fee','clean_up_cost','discount_amount','judgment_amount', 'date_diff',
                'violation_street_number','a_l','c_l','zip_code_c'
                     ,'agency_name', 'state', 'disposition'
               ]

X = df_train[selected_features]
X = pd.get_dummies(X, columns=['agency_name', 'state', 'disposition'])
y = df_train[target_column]

test = df_test[selected_features]
test = pd.get_dummies(test, columns=['agency_name', 'state', 'disposition'])

shared_features = list(set(X.columns) & set(test.columns))

X = X[shared_features]
test = test[shared_features]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

from sklearn.ensemble import RandomForestClassifier

test = scaler.transform(test)

clf = RandomForestClassifier(n_estimators=200).fit(X, y)


test.columns

#test.shape

result = clf.predict_proba(test)

In [302]:
def blight_model():
    import pandas as pd
    import numpy as np

    df_test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    df_train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    df_address =  pd.read_csv('addresses.csv')
    df_latlons = pd.read_csv('latlons.csv')

    df_address = df_address.set_index('address').join(df_latlons.set_index('address'), how='left').reset_index()
    df_train = df_train.set_index('ticket_id').join(df_address.set_index('ticket_id')).reset_index()
    df_test = df_test.set_index('ticket_id').join(df_address.set_index('ticket_id')).reset_index()


    df_train = df_train[~df_train.compliance.isnull()]
    df_train = df_train[~df_train.hearing_date.isnull()]
    df_train['date_diff'] = abs((pd.to_datetime(pd.Series(df_train.ticket_issued_date)).dt.date - \
        pd.to_datetime(pd.Series(df_train.hearing_date)).dt.date )).fillna(pd.Timedelta('-1 days')).dt.days

    df_test['date_diff'] = abs((pd.to_datetime(pd.Series(df_test.ticket_issued_date)).dt.date - \
        pd.to_datetime(pd.Series(df_test.hearing_date)).dt.date )).fillna(pd.Timedelta('-1 days')).dt.days

    df_train.lat.fillna(method='pad', inplace=True)
    df_train.lon.fillna(method='pad', inplace=True)
    df_train.state.fillna(method='pad', inplace=True)

    df_test.lat.fillna(method='pad', inplace=True)
    df_test.lon.fillna(method='pad', inplace=True)
    df_test.state.fillna(method='pad', inplace=True)
    
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    df = pd.concat([df_train['agency_name'], df_test['agency_name']], ignore_index=True)
    le.fit(df)
    df_train['a_l'] = le.transform(df_train['agency_name'])
    df_test['a_l'] = le.transform(df_test['agency_name'])
    le2 = preprocessing.LabelEncoder()
    df = pd.concat([df_train['city'], df_test['city']]).fillna(method='backfill')
    le2.fit(df)
    df_train['c_l'] = le2.transform(df_train['city'])
    df_test['c_l'] = le2.transform(df_test['city'].fillna(method='backfill'))
    df_train['violator_name'].fillna(value='ML',inplace=True)
    df_test['violator_name'].fillna(value='ML',inplace=True)
    df_train['zip_code_c'] = pd.to_numeric(df_train.zip_code, errors='coerce').fillna(method='backfill')
    df_test['zip_code_c'] = pd.to_numeric(df_test.zip_code, errors='coerce').fillna(method='backfill')   
    target_column = ['compliance']

    selected_features = ['fine_amount', 'admin_fee', 'state_fee',
               'late_fee','clean_up_cost','discount_amount','judgment_amount', 'date_diff',
                'violation_street_number','a_l','c_l','zip_code_c'
                     ,'agency_name', 'state', 'disposition'
               ]

    X = df_train[selected_features]
    X = pd.get_dummies(X, columns=['agency_name', 'state', 'disposition'])
    y = df_train[target_column]

    test = df_test[selected_features]
    test = pd.get_dummies(test, columns=['agency_name', 'state', 'disposition'])

    shared_features = list(set(X.columns) & set(test.columns))

    X = X[shared_features]
    test = test[shared_features]

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    from sklearn.ensemble import RandomForestClassifier

    test = scaler.transform(test)

    clf = RandomForestClassifier(n_estimators=200).fit(X, y)
    
    result = clf.predict_proba(test)

    output = pd.Series(data=result[:,1], index=df_test['ticket_id'], dtype="float32" )
    output.name = 'compliance'
    return output

In [304]:
#result = blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
